In [1]:
# onedork
# grade3
# oceans16
# chesterish
# monokai
# solarizedl
# solarizedd

In [2]:
!pip install jupyterthemes
!jt -t oceans16


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
print(f"es_langchain **")

es_langchain **


In [4]:
# Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitrary Python function. 
# You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features.

In [5]:
!pip install gradio
!pip install -U email-validator


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import gradio as gr
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


/Users/euiyoung.hwang/ES/Python_Workspace/Vector_DB_with_LLM/.venv/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [7]:
def user_greeting(name, intensity):
    # return "안녕하세요! " + name + "님, 첫 번째 Gradio 애플리케이션에 오신 것을 환영합니다!😎"
    return "Hello, " + name + "!" * int(intensity)

In [8]:
# You can run Gradio in your favorite code editor, Jupyter notebook, Google Colab, or anywhere else you write Python
# app = gr.Interface(fn=user_greeting, inputs="text", outputs="text")
app = gr.Interface(fn=user_greeting, inputs=["text", "slider"], outputs="text")
app.launch()
# app.launch(share=True)  # Share your demo with just 1 extra parameter 🚀

2024-12-21 15:07:59,247 : INFO : HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-21 15:07:59,318 : INFO : HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2024-12-21 15:07:59,484 : INFO : HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [9]:
from elasticsearch import Elasticsearch
import os
import json

In [10]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [11]:
''' https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/migration.html '''
''' In 8.0.0 responses are no longer the raw deserialized response body and instead an object with two properties, meta and body '''
es = Elasticsearch(hosts="http://localhost:9202", headers=get_headers(), timeout=5,  verify_certs=False)
# print(es.cat.health())
# print(type(es.cluster.health()))
resp = es.cluster.health()
# print(resp)
# print(resp['status'], resp.keys(), resp.values())
print(resp.body, type(resp.body))

# json_dict = {}
# for k, v in resp.items():
#     json_dict.update({k : v})
    
# print(json_dict, type(json_dict))
# print(json.dumps(json_dict, indent=2))
print(json.dumps(resp.body, indent=2))

/var/folders/7t/cmft4byj1n7bjh9vhd79ldyr0000gn/T/ipykernel_49320/506791230.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts="http://localhost:9203", headers=get_headers(), timeout=5,  verify_certs=False)
2024-12-21 15:07:59,835 : INFO : GET http://localhost:9203/_cluster/health [status:200 duration:0.008s]


{'cluster_name': 'docker-elasticsearch', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 71, 'active_shards': 71, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 41, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 63.39285714285714} <class 'dict'>
{
  "cluster_name": "docker-elasticsearch",
  "status": "yellow",
  "timed_out": false,
  "number_of_nodes": 1,
  "number_of_data_nodes": 1,
  "active_primary_shards": 71,
  "active_shards": 71,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 41,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 63.39285714285714
}


In [12]:
''' json to dataframe '''
import pandas as pd

df = pd.DataFrame.from_dict([resp.body])
df.head(10)

,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,docker-elasticsearch,yellow,False,1,1,71,71,0,0,41,0,0,0,0,63.392857


In [13]:
''' https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/migration.html '''
''' In 8.0.0 responses are no longer the raw deserialized response body and instead an object with two properties, meta and body '''
def es_get_health(host):
    es_client = Elasticsearch(hosts="http://{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return json.dumps(es_client.cluster.health().body, indent=2), es_client.cluster.health()['status']

In [14]:
# Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitrary Python function. 
# You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features.
''' https://www.gradio.app/guides/quickstart '''
''' https://www.gradio.app/docs/gradio/label '''
''' Prerequisite: Gradio requires Python 3.10 or higher. --> We recommend installing Gradio using pip, pip install --upgrade gradio'''
# http://127.0.0.1:7880/?__theme=dark
# app = gr.Interface(fn=es_get_health, inputs=["text"], outputs="text")
with gr.Blocks() as app:
    gr.Markdown("# ES Cluster Health")
    # es_status = json.loads(es_get_health("localhost:9201"))['status']
    label = gr.Label(value = "", show_label=True, label="ES cluster status")
    gr.Interface(fn=es_get_health, inputs=["text"], outputs=["text", label])
        
app.launch()

2024-12-21 15:08:00,075 : INFO : HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-21 15:08:00,079 : INFO : HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


2024-12-21 15:08:00,232 : INFO : HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-12-21 15:08:00,316 : INFO : HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [15]:
''' https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/ '''
''' Langchain Elasticsearch '''
!pip install langchain-elasticsearch

  Obtaining dependency information for langchain-elasticsearch from https://files.pythonhosted.org/packages/d4/0c/3d9330f0c1c6e6aba285279f4048a3cda1af30fd8a05ddaf242d59110175/langchain_elasticsearch-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for simsimd>=3 from https://files.pythonhosted.org/packages/5d/2b/9e7d42ac54bdb32d76953db3bc83eec29bd5d5c9a4069d380b18e200d6bd/simsimd-6.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 791.4 kB/s eta 0:00:0031m1.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.0 MB/s eta 0:00:000:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
